In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import datetime # manipulating date formats
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
sales=pd.read_csv("../input/sales_train.csv")

item_cat=pd.read_csv("../input/item_categories.csv")
item=pd.read_csv("../input/items.csv")
sub=pd.read_csv("../input/sample_submission.csv")
shops=pd.read_csv("../input/shops.csv")
test=pd.read_csv("../input/test.csv")

In [ ]:
sales['date'] = pd.to_datetime(sales.date, format='%d.%m.%Y')

In [ ]:
sales.info()

In [ ]:
monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])["date","item_price","item_cnt_day"].agg({"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

In [ ]:
# take a peak
monthly_sales.head(10)

In [ ]:
plt.figure(figsize=(18,6))
item.groupby('item_category_id')['item_id'].count().plot.bar()
plt.title("Items per Category")
plt.ylabel('# of items', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.show()

## Single Series

In [ ]:
Temp = sales.groupby(['date_block_num'])['item_cnt_day'].sum()
Temp.plot(figsize=(15, 5), title='Items per Category', fontsize=14)
plt.ylabel('# of items', fontsize=12)
plt.xlabel('Category', fontsize=12)

In [ ]:
plt.figure(figsize=(18, 9))
plt.plot(Temp.rolling(window=12, center=False).mean(), label='Rolling Mean')
plt.plot(Temp.rolling(window=12, center=False).std(), label='Rolling sd')
plt.legend()

In [ ]:
import statsmodels.api as sm
sm.tsa.seasonal_decompose(Temp.values, freq = 12, model='multiplicative').plot()
result = sm.tsa.stattools.adfuller(Temp.values)
plt.figure(figsize=(15, 8))
plt.show()

In [ ]:
import statsmodels.api as sm
sm.tsa.seasonal_decompose(Temp.values, freq = 12, model='addictive').plot()
result = sm.tsa.stattools.adfuller(Temp.values)
plt.figure(figsize=(15, 8))
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(24).mean() # 24 hours on each day
    rolstd = timeseries.rolling(24).std()
    
    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
test_stationarity(Temp)

In [ ]:
# to remove trend
from pandas import Series as Series
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# invert differenced forecast
def inverse_difference(last_ob, value):
    return value + last_ob

In [ ]:
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,16))
plt.subplot(311)
plt.title('Original')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts)
plt.subplot(312)
plt.title('After De-trend')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts)
plt.plot(new_ts)
plt.plot()

plt.subplot(313)
plt.title('After De-seasonalization')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts,12)       # assuming the seasonality is 12 months long
plt.plot(new_ts)
plt.plot()

In [ ]:
# now testing the stationarity again after de-seasonality
test_stationarity(new_ts)